In [7]:
import os
from util import labeledImage
from azureapi import AzureCVObjectDetectionAPI
from sklearn.model_selection import train_test_split

In [38]:
# collate labels #
img_dir = './data/mos2/'
labels_dir = './data/labels/'

labeled_images = []
tag = 'molecule'

# filter img fns to get only 22-28.png #
all_img_fns = os.listdir(img_dir)
img_fns = [fn for fn in all_img_fns if int(fn.split('.')[0]) in list(range(1,29))]

for file in img_fns:
    if file.endswith(".png"):
        image_path = os.path.join(img_dir, file)
        label_path = os.path.join(labels_dir, file.split('.')[0] + '.txt')
        labeled_images.append(labeledImage(image_path))
        labeled_images[-1].add_labels_from_file(tag, label_path)

In [37]:
print(labeled_images[0])

Labeled image 28.png
    location: ./data/mos2/28.png
    shape: (386, 337, 3)
    labels:
    - molecule: 
      [127, 73, 55, 61]
      [250, 142, 30, 30]
      [127, 178, 35, 29]
      [40, 113, 56, 46]
      [0, 176, 13, 24]
      [7, 0, 36, 18]
      [286, 8, 51, 47]
      [227, 319, 77, 67]
      [325, 359, 12, 27]



In [14]:
# split images into train and test sets #
train_images, test_images = train_test_split(labeled_images, test_size=0.2, random_state=42)

train_image_numbers = [int(img.name.split('.')[0]) for img in train_images]
test_image_numbers = [int(img.name.split('.')[0]) for img in test_images]
print(f'Training Images: {train_image_numbers}')
print(f'Testing Images: {test_image_numbers}')

Training Images: [14, 2, 12, 7, 5, 21, 11, 18, 9, 13, 20, 16, 19, 3, 8, 15]
Testing Images: [1, 6, 4, 10, 17]


In [15]:
# create AzureAPI object and upload images #
Endpoint    = 'https://southcentralus.api.cognitive.microsoft.com/'
Key         = 'dc7af5f887c84f90a51d25549c7ad863'
Resource_Id = '/subscriptions/cbcd0d6a-0ae5-48a7-a46d-4a4f3c692c71/resourceGroups/rg-amlclass-longnic/providers/Microsoft.CognitiveServices/accounts/cv_longnic'
Project_Id  = '5ce9e36e-942f-45c5-a102-837662335aa6'

ACVObjectDetector = AzureCVObjectDetectionAPI(Endpoint, Key, Resource_Id, Project_Id)
ACVObjectDetector.upload_training_images(train_images)

Adding images...
Adding images: batch  1
Image batch upload failed.
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
Image status:  OKDuplicate
